# Validation

In [95]:
import pandas as pd
import numpy as np
import json
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
import sys
import pickle
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

# load environment variables
load_dotenv()

#add working directory to sys path to execute utils/dataset.py
working_dir = os.environ.get("WORKING_DIRECTORY")
sys.path.insert(0, working_dir)

from utils.pipeline_moduls import outlier_label, outlier_num


ModuleNotFoundError: No module named 'tabulate'

In [ ]:
#TODO: Hier kommmen die Daten von Herrn Prof. Dr. Meth
df = pd.read_csv("../wine_training.csv", sep=";", index_col=0)
df.head()

,wine type,fixed acidity,volatile acidity,citric acid,residual sugar,magnesium,flavanoids,minerals,calcium,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,Pinot noir,5.8,0.15,0.49,1.1,76.729301,894.94,186.639301,109.91,0.048,21.0,98.0,0.99290,3.19,0.48,9.2,5
1,Merlot,6.6,0.25,0.32,5.6,4.795712,1160.95,251.875712,247.08,0.039,15.0,68.0,0.99163,2.96,0.52,11.1,6
2,Chardonnay,6.7,0.21,0.34,1.5,85.193710,789.82,304.703710,219.51,0.035,45.0,123.0,0.98949,3.24,0.36,12.6,7
3,Merlot,8.3,0.28,0.27,17.5,11.976525,777.86,237.586525,225.61,0.045,48.0,253.0,1.00014,3.02,0.56,9.1,6
4,Merlot,7.5,0.42,0.19,6.9,5.599673,785.72,95.399673,89.80,0.041,62.0,150.0,0.99508,3.23,0.37,10.0,6


In [ ]:
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=[np.number]).columns
#drop 'quality' from numerical features (its a series)
numerical_features = numerical_features.drop('quality')
label = pd.Series('quality')

## Cleaning Pipeline

### Methods

In [ ]:
outlier_detection_label = FunctionTransformer(outlier_label).set_output(transform="pandas")
outlier_detection = FunctionTransformer(outlier_num).set_output(transform="pandas")

def feature_selection(df, json_file: str = "./task_2/dropped_features.json"):
    try:
        with open(json_file, 'r') as f:
            dropped_features = json.load(f)
    except FileNotFoundError:
        print(f"File {json_file} not found")
        return None

    # Find common features between DataFrame columns and dropped features
    common_features = list(set(df.columns).intersection(dropped_features))
    if not common_features:
        print("No dropped features found in the DataFrame.")
        return df
    
    # Remove the common features from the DataFrame
    df = df.drop(common_features, axis=1)
    print("Working with following columns:", df.columns.values)
    return df
feature_selection = FunctionTransformer(feature_selection).set_output(transform="pandas")

/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


#### Sub-Pipeline: Categorical Features

In [ ]:
categorical_imputer = SimpleImputer(strategy="most_frequent").set_output(transform="pandas")

#pipeline for categorical features
categorical_pipeline = Pipeline(steps=[])
categorical_pipeline.steps.append(('imputer', categorical_imputer))
categorical_pipeline.steps.append(('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform="pandas")))

#### Sub-Pipeline: Numerical Features

In [ ]:
#pipeline for numerical features
numeric_pipeline = Pipeline(steps=[])
numerical_imputer = SimpleImputer(strategy="mean").set_output(transform="pandas")

numeric_pipeline.steps.append(('imputer', numerical_imputer))
numeric_pipeline.steps.append(('outlier_detection', outlier_detection))

#### Sub-Pipeline: Label

In [ ]:
#pipeline for label
label_pipeline = Pipeline(steps=[])
label_pipeline.steps.append(('imputer', numerical_imputer))


### Pipeine

In [ ]:
cleaning_pipeline = Pipeline(steps=[
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features),
        ('label', label_pipeline, label)
    ]).set_output(transform="pandas")
cleaning_pipeline.steps.append(('preprocessor', preprocessor))
cleaning_pipeline.steps.append(("outlier_detection_label", outlier_detection_label))
cleaning_pipeline.steps.append(('feature_selection', feature_selection))
cleaning_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('outlier_detection',
                                                                   FunctionTransformer(func=<function outlier_num at 0x13f0e1c60>))]),
                                                  Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'magnesium', 'flavanoids', 'minerals', 'calcium', 'chlorides',
       'free sulfur dioxide', 'to...
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['wine type'], dtype='object')),
                                                 ('label',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  0    quality
dtype: object)])),
                ('outlier_detection_label',
                 FunctionTransformer(func=<function outlier_label at 0x13e1b6660>)),
                ('feature_selection',
                 FunctionTransformer(func=<function feature_selection at 0x13fcc58a0>))])

In [ ]:
def run_all(df):
    cleaned_data = cleaning_pipeline.fit_transform(df)
    #load model with pickle
    model = pickle.load(open('./task_2/best_model__random_forest.pkl', 'rb'))
    X = cleaned_data.drop('label__quality', axis=1)
    y = cleaned_data['label__quality']
    print("----------------------------------------")
    print("Accuracy:", model.score(X, y))
    print("----------------------------------------")
    

run_all(df)

Working with following columns: ['num__fixed acidity' 'num__volatile acidity' 'num__citric acid'
 'num__magnesium' 'num__flavanoids' 'num__minerals' 'num__chlorides'
 'num__total sulfur dioxide' 'num__density' 'num__pH' 'num__sulphates'
 'num__alcohol' 'cat__wine type_Cabernet Sauvignon'
 'cat__wine type_Chardonnay' 'cat__wine type_Gamay'
 'cat__wine type_Merlot' 'label__quality']
----------------------------------------
Accuracy: 0.9676183145919519
----------------------------------------
